In [11]:
%%html
<table cellspacing='1'>
<td>
    <td> <img src="./pyseal_concept_and_usage.JPG" alt="image" style="width: 1500px;"/> </td>

</table>

# query without password as encryption key

In [3]:
from base64 import urlsafe_b64encode, urlsafe_b64decode

def encode(data):
    return urlsafe_b64encode(bytes(data, 'utf-8'))

def decode(enc):
    return urlsafe_b64decode(enc).decode()


In [14]:
myencrypted_str= encode('SELECT pid,age,gender from MYDB')
print("encrypted string without password : \n",myencrypted_str ,"\n")
print("decoded string without password ;\n",decode(myencrypted_str)) 

encrypted string without password : 
 b'U0VMRUNUIHBpZCxhZ2UsZ2VuZGVyIGZyb20gTVlEQg==' 

decoded string without password ;
 SELECT pid,age,gender from MYDB


# encrypt your query with password as keys 

In [5]:
import base64
from Crypto.Cipher import AES
from Crypto.Hash import SHA256
from Crypto import Random

def encrypt(key, source, encode=True):
    key = SHA256.new(key).digest()  # use SHA-256 over our key to get a proper-sized AES key
    IV = Random.new().read(AES.block_size)  # generate IV
    encryptor = AES.new(key, AES.MODE_CBC, IV)
    padding = AES.block_size - len(source) % AES.block_size  # calculate needed padding
    source += bytes([padding]) * padding  # Python 2.x: source += chr(padding) * padding
    data = IV + encryptor.encrypt(source)  # store the IV at the beginning and encrypt
    return base64.b64encode(data).decode("latin-1") if encode else data

def decrypt(key, source, decode=True):
    if decode:
        source = base64.b64decode(source.encode("latin-1"))
    key = SHA256.new(key).digest()  # use SHA-256 over our key to get a proper-sized AES key
    IV = source[:AES.block_size]  # extract the IV from the beginning
    decryptor = AES.new(key, AES.MODE_CBC, IV)
    data = decryptor.decrypt(source[AES.block_size:])  # decrypt
    padding = data[-1]  # pick the padding value from the end; Python 2.x: ord(data[-1])
    if data[-padding:] != bytes([padding]) * padding:  # Python 2.x: chr(padding) * padding
        raise ValueError("Invalid padding...")
    return data[:-padding]  # remove the padding

# encryption twice to ensure no-one can temper with your query

In [18]:
my_password = b"my_secret_passw0rd1"
my_data = b"SELECT id ,age, gender ,medicine from myDB"

print("key:  {} \n".format(my_password))
print("original query string with password used in encryption : {} \n".format(my_data))
encrypted = encrypt(my_password, my_data)
print("\nencrypted string:  {}\n".format(encrypted))
decrypted = decrypt(my_password, encrypted)

#print("\ndata match: {}\n".format(my_data == decrypted))
print("\nSecond round to double encryption original message....\n")
encrypted = encrypt(my_password, my_data)
print("\nencrypted string:  {}\n".format(encrypted))
decrypted = decrypt(my_password, encrypted)

#print("\ndata match: {}\n".format(my_data == decrypted))

key:  b'my_secret_passw0rd1' 

original query string with password used in encryption : b'SELECT id ,age, gender ,medicine from myDB' 


encrypted string:  w5doiLOc9PL985JUM2kw3tXPPF0T4EsDYGIHqDjCklHTNw9q2qqEcPMDheDO7574TLtG/VyGkyDcffVNianoIQ==


Second round to double encryption original message....


encrypted string:  lNZLGb2SEMRGpI1F9ZSvVsDMsPO6OF/QAk3857XAoUBDUUgbAzuG98c4O9PhAIhsn8/KVVLJPLaHDQ6cN5oNaw==

